In [1]:
!nvidia-smi

Sat Jan 25 14:12:04 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
%%writefile tiledMatrixMult.cu

#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#include <time.h>
#include <cuda_runtime.h>

#define TILE_WIDTH 32

inline cudaError_t checkCuda(cudaError_t result) {
    if (result != cudaSuccess) {
        fprintf(stderr, "Cuda Runtime Error: %s\n", cudaGetErrorString(result));
        assert(result == cudaSuccess);
    }
    return result;
}

__global__ void tiledMatrixMultKernel(float *M, float *N, float *P, int width) {
    __shared__ float M_s[TILE_WIDTH][TILE_WIDTH];
    __shared__ float N_s[TILE_WIDTH][TILE_WIDTH];

    int bx = blockIdx.x;   int by = blockIdx.y;
    int tx = threadIdx.x;  int ty = threadIdx.y;

    // Identify the row and column of the P element to work on
    int row = by * TILE_WIDTH + ty;
    int col = bx * TILE_WIDTH + tx;

    // Loop over the M and N tiles required to compute P element
    float Pvalue = 0;
    for (int tile = 0; tile < (width + TILE_WIDTH - 1) / TILE_WIDTH; tile++) {

        // Collaborative loading of M and N tiles into shared memory
        if (row < width && (tile * TILE_WIDTH + tx) < width) {
            M_s[ty][tx] = M[row * width + (tile * TILE_WIDTH + tx)];
        } else {
            M_s[ty][tx] = 0.0f;  // Zero-padding for out-of-bounds accesses
        }

        if ((tile * TILE_WIDTH + ty) < width && col < width) {
            N_s[ty][tx] = N[(tile * TILE_WIDTH + ty) * width + col];
        } else {
            N_s[ty][tx] = 0.0f;  // Zero-padding for out-of-bounds accesses
        }
        __syncthreads();

        // Compute the partial product for the tile
        for (int k = 0; k < TILE_WIDTH; k++) {
            Pvalue += M_s[ty][k] * N_s[k][tx];
        }
        __syncthreads();
    }

    // Write the result to the output matrix P
    if (row < width && col < width) {
        P[row * width + col] = Pvalue;
    }
}

void tiledMatrixMult(float *M_h, float *N_h, float *P_h, int width) {
    // Allocate GPU memory
    float *M_d, *N_d, *P_d;
    checkCuda( cudaMalloc((void**)&M_d, width * width * sizeof(float)) );
    checkCuda( cudaMalloc((void**)&N_d, width * width * sizeof(float)) );
    checkCuda( cudaMalloc((void**)&P_d, width * width * sizeof(float)) );

    // Transfer data host -> device
    checkCuda( cudaMemcpy(M_d, M_h, width * width * sizeof(float), cudaMemcpyHostToDevice) );
    checkCuda( cudaMemcpy(N_d, N_h, width * width * sizeof(float), cudaMemcpyHostToDevice) );

    // Perform mmult
    dim3 numThreadsPerBlock(32, 32);
    dim3 numBlocks((width + numThreadsPerBlock.x - 1) / numThreadsPerBlock.x,
                   (width + numThreadsPerBlock.y - 1) / numThreadsPerBlock.y
    );
    tiledMatrixMultKernel<<< numBlocks, numThreadsPerBlock >>>(M_d, N_d, P_d, width);
    checkCuda( cudaGetLastError() );
    checkCuda( cudaDeviceSynchronize() );

    // Transfer data device -> host
    checkCuda( cudaMemcpy(P_h, P_d, width * width * sizeof(float), cudaMemcpyDeviceToHost) );

    // Free GPU memory
    checkCuda( cudaFree(M_d) );
    checkCuda( cudaFree(N_d) );
    checkCuda( cudaFree(P_d) );
}

int main(void) {
    srand(time(NULL));

    // N x N matrix
    int n = 1 << 12;

    float *M = (float*) malloc(n * n * sizeof(float));
    float *N = (float*) malloc(n * n * sizeof(float));
    float *P = (float*) malloc(n * n * sizeof(float));

    if(!M || !N || !P) {
        fprintf(stderr, "Malloc Error.\n");
        return 1;
    }

    for (int i = 0; i < n * n; i++) {
        M[i] = rand() / (float)RAND_MAX;
        N[i] = rand() / (float)RAND_MAX;
    }

    tiledMatrixMult(M, N, P, n);

    printf("P[0] = %f | Expected:", P[0]);
    float sum = 0;
    for (int k = 0; k < n; k++) {
        sum += M[k] * N[k * n];
    }
    printf("%f\n", sum);


    free(M);
    free(N);
    free(P);

    return 0;
}

Overwriting tiledMatrixMult.cu


In [7]:
%%shell
nvcc tiledMatrixMult.cu -o tiledMatrixMult
nvprof ./tiledMatrixMult

==1284== NVPROF is profiling process 1284, command: ./tiledMatrixMult
P[0] = 1011.952698 | Expected:1011.952698
==1284== Profiling application: ./tiledMatrixMult
==1284== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   77.21%  249.63ms         1  249.63ms  249.63ms  249.63ms  tiledMatrixMultKernel(float*, float*, float*, int)
                   14.08%  45.533ms         1  45.533ms  45.533ms  45.533ms  [CUDA memcpy DtoH]
                    8.71%  28.167ms         2  14.083ms  13.987ms  14.179ms  [CUDA memcpy HtoD]
      API calls:   58.84%  249.64ms         1  249.64ms  249.64ms  249.64ms  cudaDeviceSynchronize
                   22.46%  95.294ms         3  31.765ms  144.60us  94.994ms  cudaMalloc
                   17.79%  75.491ms         3  25.164ms  14.209ms  46.924ms  cudaMemcpy
                    0.82%  3.4651ms         3  1.1550ms  197.05us  2.1414ms  cudaFree
                    0.05%  220.27us         1  2

In [8]:
%%writefile coarsedTileMatrixMultKernel.cu

#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#include <time.h>
#include <cuda_runtime.h>

#define TILE_WIDTH 32
#define COARSE_FACTOR 4

inline cudaError_t checkCuda(cudaError_t result) {
    if (result != cudaSuccess) {
        fprintf(stderr, "Cuda Runtime Error: %s\n", cudaGetErrorString(result));
        assert(result == cudaSuccess);
    }
    return result;
}

__global__ void coarsedTiledMatrixMultKernel(float *M, float *N, float *P, int width) {
    __shared__ float M_s[TILE_WIDTH][TILE_WIDTH];
    __shared__ float N_s[TILE_WIDTH][TILE_WIDTH];

    int bx = blockIdx.x;   int by = blockIdx.y;
    int tx = threadIdx.x;  int ty = threadIdx.y;

    // Identify the row and column of the P element to work on
    int row = by * TILE_WIDTH + ty;
    int colStart = bx * TILE_WIDTH * COARSE_FACTOR + tx;

    // Initialize Pvalue for all output elements
    float Pvalue[COARSE_FACTOR];
    for (int c = 0; c < COARSE_FACTOR; c++) {
        Pvalue[c] = 0.0f;
    }

    // Loop over the M and N tiles required to compute P element
    for (int tile = 0; tile < width/TILE_WIDTH; tile++) {

        // Collaborative loading of M tile into shared memory
        M_s[ty][tx] = M[row * width + tile*TILE_WIDTH + tx];

        for (int c = 0; c < COARSE_FACTOR; c++) {
            int col = colStart + c * TILE_WIDTH;

            // Collaborative loading of N tile into shared memory
            N_s[ty][tx] = N[(tile*TILE_WIDTH + ty) * width + col];
            __syncthreads();

            for (int k = 0; k < TILE_WIDTH; k++) {
                Pvalue[c] += M_s[ty][k] * N_s[k][tx];
            }
            __syncthreads();
        }
    }

    for (int c = 0; c < COARSE_FACTOR; c++) {
        int col = colStart + c * TILE_WIDTH;
        P[row * width + col] = Pvalue[c];
    }

}

void tiledMatrixMult(float *M_h, float *N_h, float *P_h, int width) {
    // Allocate GPU memory
    float *M_d, *N_d, *P_d;
    checkCuda( cudaMalloc((void**)&M_d, width * width * sizeof(float)) );
    checkCuda( cudaMalloc((void**)&N_d, width * width * sizeof(float)) );
    checkCuda( cudaMalloc((void**)&P_d, width * width * sizeof(float)) );

    // Transfer data host -> device
    checkCuda( cudaMemcpy(M_d, M_h, width * width * sizeof(float), cudaMemcpyHostToDevice) );
    checkCuda( cudaMemcpy(N_d, N_h, width * width * sizeof(float), cudaMemcpyHostToDevice) );

    // Perform mmult
    dim3 numThreadsPerBlock(32, 32);
    dim3 numBlocks((width + numThreadsPerBlock.x * COARSE_FACTOR - 1) / (numThreadsPerBlock.x * COARSE_FACTOR),
                   (width + numThreadsPerBlock.y - 1) / numThreadsPerBlock.y
    );
    coarsedTiledMatrixMultKernel<<< numBlocks, numThreadsPerBlock >>>(M_d, N_d, P_d, width);
    checkCuda( cudaGetLastError() );
    checkCuda( cudaDeviceSynchronize() );

    // Transfer data device -> host
    checkCuda( cudaMemcpy(P_h, P_d, width * width * sizeof(float), cudaMemcpyDeviceToHost) );

    // Free GPU memory
    checkCuda( cudaFree(M_d) );
    checkCuda( cudaFree(N_d) );
    checkCuda( cudaFree(P_d) );
}

int main(void) {
    srand(time(NULL));

    // N x N matrix
    int n = 1 << 12;

    float *M = (float*) malloc(n * n * sizeof(float));
    float *N = (float*) malloc(n * n * sizeof(float));
    float *P = (float*) malloc(n * n * sizeof(float));

    if(!M || !N || !P) {
        fprintf(stderr, "Malloc Error.\n");
        return 1;
    }

    for (int i = 0; i < n * n; i++) {
        M[i] = rand() / (float)RAND_MAX;
        N[i] = rand() / (float)RAND_MAX;
    }

    tiledMatrixMult(M, N, P, n);

    printf("P[0] = %f | Expected:", P[0]);
    float sum = 0;
    for (int k = 0; k < n; k++) {
        sum += M[k] * N[k * n];
    }
    printf("%f\n", sum);


    free(M);
    free(N);
    free(P);

    return 0;
}

Overwriting coarsedTileMatrixMultKernel.cu


In [9]:
%%shell
nvcc coarsedTileMatrixMultKernel.cu -o coarsedTileMatrixMultKernel
nvprof ./coarsedTileMatrixMultKernel

==1412== NVPROF is profiling process 1412, command: ./coarsedTileMatrixMultKernel
P[0] = 1012.294006 | Expected:1012.294006
==1412== Profiling application: ./coarsedTileMatrixMultKernel
==1412== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   74.38%  245.73ms         1  245.73ms  245.73ms  245.73ms  coarsedTiledMatrixMultKernel(float*, float*, float*, int)
                   16.61%  54.862ms         1  54.862ms  54.862ms  54.862ms  [CUDA memcpy DtoH]
                    9.01%  29.778ms         2  14.889ms  14.563ms  15.216ms  [CUDA memcpy HtoD]
      API calls:   55.03%  245.73ms         1  245.73ms  245.73ms  245.73ms  cudaDeviceSynchronize
                   24.81%  110.81ms         3  36.938ms  205.42us  110.40ms  cudaMalloc
                   19.46%  86.891ms         3  28.964ms  14.784ms  56.654ms  cudaMemcpy
                    0.58%  2.6029ms         3  867.64us  323.92us  1.1783ms  cudaFree
                 

For N = 1 << 10 (1024 x 1024): Thread coarsening is not worth it because the thread blocks are not being serialized by the hardware. Different thread blocks loading the same input tile is redundant, but the price we pay here is worth it. Our matrices are too small.

For N = 1 << 12  (4096 x 4096): Thread coarsening is worth it because the blocks are being serialized by the hardware. This way, the coarsened thread block loads the input tiles of M once and reuse them for multiple output tiles.